In [1]:
%%capture
!pip install --no-index --find-links=/kaggle/input/reranker-scripts/packages -U transformers bitsandbytes accelerate peft

In [2]:
import re
import pandas as pd
import numpy as np

comp_dir = '/kaggle/input/eedi-mining-misconceptions-in-mathematics'

train          = pd.read_csv(f'{comp_dir}/train.csv')
test           = pd.read_csv(f'{comp_dir}/test.csv')
misconceptions = pd.read_csv(f'{comp_dir}/misconception_mapping.csv')

In [3]:
zeros = [5,
 7,
 10,
 12,
 13,
 17,
 18,
 24,
 25,
 30,
 40,
 41,
 43,
 46,
 50,
 53,
 56,
 59,
 62,
 63,
 65,
 67,
 69,
 72,
 73,
 75,
 79,
 80,
 87,
 90,
 92,
 93,
 94,
 96,
 104,
 112,
 115,
 116,
 121,
 122,
 124,
 125,
 128,
 129,
 135,
 139,
 140,
 144,
 145,
 147,
 148,
 149,
 150,
 155,
 156,
 157,
 160,
 163,
 165,
 168,
 170,
 173,
 174,
 175,
 177,
 178,
 180,
 184,
 188,
 192,
 193,
 198,
 199,
 200,
 203,
 204,
 206,
 208,
 209,
 214,
 215,
 216,
 222,
 225,
 231,
 235,
 237,
 238,
 240,
 241,
 243,
 246,
 250,
 254,
 257,
 258,
 259,
 266,
 269,
 274,
 275,
 276,
 277,
 286,
 288,
 291,
 298,
 299,
 301,
 304,
 310,
 311,
 313,
 314,
 316,
 319,
 320,
 321,
 323,
 324,
 325,
 327,
 330,
 333,
 335,
 336,
 341,
 342,
 343,
 348,
 351,
 354,
 356,
 358,
 359,
 360,
 361,
 365,
 367,
 368,
 369,
 371,
 385,
 386,
 387,
 390,
 394,
 395,
 399,
 403,
 406,
 407,
 410,
 411,
 412,
 413,
 416,
 419,
 420,
 425,
 428,
 429,
 430,
 431,
 432,
 434,
 435,
 437,
 440,
 442,
 444,
 448,
 453,
 456,
 457,
 459,
 462,
 465,
 472,
 473,
 475,
 476,
 477,
 479,
 480,
 482,
 484,
 485,
 487,
 489,
 490,
 494,
 497,
 500,
 502,
 504,
 505,
 506,
 513,
 514,
 516,
 517,
 518,
 522,
 523,
 529,
 530,
 534,
 535,
 536,
 538,
 541,
 543,
 546,
 548,
 552,
 555,
 559,
 560,
 561,
 562,
 569,
 571,
 574,
 575,
 579,
 580,
 582,
 586,
 592,
 593,
 595,
 596,
 597,
 598,
 605,
 607,
 610,
 612,
 613,
 615,
 622,
 627,
 632,
 634,
 636,
 640,
 645,
 647,
 660,
 662,
 665,
 667,
 675,
 676,
 677,
 678,
 679,
 681,
 682,
 683,
 689,
 692,
 693,
 696,
 697,
 698,
 700,
 701,
 703,
 705,
 707,
 714,
 716,
 720,
 721,
 722,
 726,
 728,
 731,
 738,
 740,
 741,
 744,
 748,
 749,
 750,
 752,
 753,
 755,
 761,
 763,
 768,
 769,
 770,
 771,
 774,
 775,
 776,
 777,
 778,
 781,
 786,
 787,
 788,
 798,
 799,
 802,
 803,
 805,
 810,
 817,
 818,
 819,
 822,
 824,
 825,
 826,
 827,
 830,
 831,
 837,
 841,
 844,
 846,
 849,
 850,
 853,
 854,
 855,
 856,
 857,
 859,
 861,
 862,
 865,
 869,
 873,
 881,
 883,
 884,
 885,
 886,
 887,
 889,
 892,
 896,
 897,
 899,
 903,
 908,
 913,
 916,
 917,
 918,
 919,
 920,
 921,
 928,
 929,
 934,
 938,
 939,
 943,
 945,
 949,
 953,
 956,
 957,
 958,
 963,
 966,
 967,
 970,
 973,
 979,
 985,
 992,
 993,
 995,
 998,
 999,
 1000,
 1001,
 1002,
 1003,
 1005,
 1006,
 1010,
 1011,
 1014,
 1015,
 1019,
 1020,
 1021,
 1023,
 1027,
 1034,
 1037,
 1038,
 1039,
 1042,
 1045,
 1052,
 1053,
 1056,
 1059,
 1060,
 1061,
 1062,
 1064,
 1065,
 1067,
 1076,
 1077,
 1080,
 1081,
 1084,
 1085,
 1086,
 1087,
 1088,
 1090,
 1092,
 1094,
 1095,
 1096,
 1097,
 1101,
 1105,
 1106,
 1109,
 1111,
 1112,
 1117,
 1121,
 1122,
 1125,
 1127,
 1129,
 1131,
 1136,
 1137,
 1141,
 1143,
 1147,
 1149,
 1154,
 1159,
 1160,
 1161,
 1162,
 1170,
 1171,
 1174,
 1178,
 1182,
 1185,
 1189,
 1191,
 1194,
 1195,
 1197,
 1199,
 1200,
 1201,
 1211,
 1217,
 1219,
 1220,
 1221,
 1227,
 1228,
 1232,
 1236,
 1238,
 1239,
 1241,
 1242,
 1243,
 1245,
 1246,
 1247,
 1249,
 1253,
 1254,
 1267,
 1273,
 1275,
 1276,
 1279,
 1281,
 1284,
 1285,
 1286,
 1289,
 1296,
 1297,
 1298,
 1299,
 1300,
 1301,
 1305,
 1309,
 1314,
 1315,
 1317,
 1323,
 1325,
 1328,
 1330,
 1331,
 1335,
 1337,
 1339,
 1341,
 1342,
 1343,
 1345,
 1346,
 1347,
 1351,
 1352,
 1353,
 1355,
 1359,
 1366,
 1368,
 1369,
 1372,
 1375,
 1377,
 1378,
 1381,
 1382,
 1385,
 1390,
 1391,
 1395,
 1397,
 1401,
 1404,
 1405,
 1407,
 1409,
 1412,
 1413,
 1414,
 1423,
 1434,
 1438,
 1439,
 1441,
 1446,
 1448,
 1451,
 1454,
 1462,
 1463,
 1465,
 1466,
 1472,
 1474,
 1475,
 1476,
 1477,
 1478,
 1482,
 1485,
 1486,
 1489,
 1493,
 1496,
 1497,
 1498,
 1501,
 1502,
 1503,
 1504,
 1506,
 1512,
 1515,
 1518,
 1521,
 1531,
 1538,
 1544,
 1545,
 1546,
 1548,
 1551,
 1555,
 1562,
 1563,
 1564,
 1565,
 1567,
 1569,
 1570,
 1573,
 1574,
 1575,
 1576,
 1578,
 1579,
 1580,
 1581,
 1583,
 1584,
 1587,
 1589,
 1592,
 1594,
 1603,
 1607,
 1612,
 1615,
 1617,
 1618,
 1620,
 1625,
 1627,
 1629,
 1634,
 1638,
 1643,
 1647,
 1649,
 1650,
 1653,
 1654,
 1662,
 1665,
 1679,
 1681,
 1683,
 1684,
 1685,
 1688,
 1689,
 1692,
 1698,
 1699,
 1700,
 1709,
 1711,
 1712,
 1713,
 1719,
 1722,
 1723,
 1727,
 1728,
 1729,
 1732,
 1736,
 1739,
 1742,
 1745,
 1747,
 1750,
 1753,
 1754,
 1758,
 1762,
 1765,
 1768,
 1770,
 1772,
 1774,
 1777,
 1778,
 1780,
 1785,
 1789,
 1797,
 1798,
 1799,
 1801,
 1802,
 1804,
 1808,
 1812,
 1816,
 1817,
 1821,
 1822,
 1826,
 1827,
 1829,
 1830,
 1833,
 1838,
 1843,
 1844,
 1848,
 1853,
 1855,
 1856,
 1857,
 1859,
 1865,
 1867,
 1873,
 1874,
 1878,
 1879,
 1888,
 1890,
 1891,
 1892,
 1899,
 1901,
 1903,
 1904,
 1905,
 1910,
 1913,
 1919,
 1922,
 1926,
 1928,
 1930,
 1931,
 1933,
 1934,
 1935,
 1936,
 1938,
 1941,
 1943,
 1947,
 1949,
 1951,
 1952,
 1955,
 1961,
 1962,
 1964,
 1967,
 1969,
 1974,
 1975,
 1977,
 1979,
 1983,
 1991,
 1998,
 2000,
 2001,
 2004,
 2005,
 2008,
 2010,
 2013,
 2014,
 2015,
 2018,
 2019,
 2025,
 2028,
 2029,
 2033,
 2036,
 2040,
 2041,
 2052,
 2057,
 2058,
 2059,
 2060,
 2061,
 2062,
 2063,
 2064,
 2065,
 2067,
 2070,
 2074,
 2075,
 2076,
 2079,
 2086,
 2088,
 2089,
 2096,
 2097,
 2099,
 2106,
 2108,
 2115,
 2116,
 2118,
 2120,
 2129,
 2136,
 2141,
 2144,
 2146,
 2151,
 2153,
 2158,
 2164,
 2165,
 2167,
 2168,
 2169,
 2170,
 2171,
 2172,
 2174,
 2176,
 2177,
 2182,
 2183,
 2184,
 2186,
 2188,
 2194,
 2195,
 2196,
 2200,
 2201,
 2202,
 2205,
 2207,
 2211,
 2213,
 2216,
 2219,
 2222,
 2223,
 2224,
 2227,
 2229,
 2232,
 2233,
 2235,
 2236,
 2242,
 2243,
 2244,
 2246,
 2247,
 2249,
 2253,
 2254,
 2255,
 2257,
 2258,
 2259,
 2267,
 2268,
 2272,
 2274,
 2276,
 2277,
 2278,
 2281,
 2283,
 2287,
 2290,
 2293,
 2294,
 2295,
 2298,
 2299,
 2300,
 2304,
 2310,
 2313,
 2315,
 2322,
 2323,
 2324,
 2325,
 2328,
 2335,
 2337,
 2338,
 2339,
 2342,
 2347,
 2348,
 2358,
 2360,
 2366,
 2367,
 2369,
 2370,
 2372,
 2379,
 2381,
 2382,
 2383,
 2385,
 2387,
 2390,
 2391,
 2393,
 2394,
 2396,
 2400,
 2403,
 2404,
 2405,
 2406,
 2407,
 2409,
 2410,
 2411,
 2418,
 2419,
 2420,
 2421,
 2422,
 2429,
 2430,
 2431,
 2434,
 2439,
 2440,
 2441,
 2444,
 2448,
 2449,
 2451,
 2452,
 2455,
 2459,
 2460,
 2462,
 2463,
 2467,
 2468,
 2471,
 2477,
 2478,
 2482,
 2483,
 2486,
 2490,
 2495,
 2496,
 2497,
 2498,
 2502,
 2503,
 2506,
 2509,
 2513,
 2518,
 2521,
 2522,
 2523,
 2526,
 2527,
 2528,
 2529,
 2533,
 2534,
 2536,
 2538,
 2540,
 2541,
 2545,
 2548,
 2552,
 2553,
 2556,
 2557,
 2559,
 2560,
 2562,
 2564,
 2567,
 2568,
 2570,
 2571,
 2574,
 2575,
 2578,
 2580,
 2582,
 2584]

# preprocessing

In [4]:
answer_cols         = ["AnswerAText", "AnswerBText", "AnswerCText", "AnswerDText"]
keep_cols           = ["QuestionId", "CorrectAnswer", "ConstructName", "SubjectName", "QuestionText" ]

def wide_to_long(df: pd.DataFrame) -> pd.DataFrame:
    answers_df = pd.melt(
        id_vars=keep_cols,
        frame=df[keep_cols + answer_cols],
        var_name='Answer', value_name='AnswerText'
    ).sort_values(["QuestionId", "Answer"]).reset_index(drop=True)    
    return answers_df

def preprocess_text(x):
    x = re.sub(r"http\w+", '', x)   # Delete URL
    x = re.sub(r"\.+", ".", x)      # Replace consecutive periods with a single period
    x = re.sub(r"\,+", ",", x)      # Replace consecutive commas with a single comma
    x = re.sub(r"\\\\", r"\\", x)   # Normalize multiple backslashes to double backslashes
    x = re.sub(r"[ ]{2,}", " ", x)  # Replace multiple spaces with a single space
    x = x.strip()                   # Remove empty characters at the beginning and end
    return x

task_description = 'Given a math question and a misconcepte incorrect answer, please retrieve the most accurate reason for the misconception.'
def get_text(row):
    text = f'''Instruct: {task_description}

Query:
###Construct###:{row['ConstructName']}
###Subject###:{row['SubjectName']}
###Question###:{row['QuestionText']}
###Correct Answer###:{row['CorrectAnswerText']}
###Incorrect Answer###:{row['AnswerText']}
###Misconception###:
'''
    return preprocess_text(text)

In [5]:
df = wide_to_long(test)
df['AnswerId'] = df.Answer.str.replace('Answer', '').str.replace('Text', '')

ca_map_df = df[df['CorrectAnswer']==df['AnswerId']][['QuestionId', 'AnswerText']].reset_index(drop=True)
ca_map_df.columns = ['QuestionId', 'CorrectAnswerText']
df = pd.merge(df, ca_map_df, on='QuestionId', how='left')

df = df[df['CorrectAnswer']!=df['AnswerId']].reset_index(drop=True)
df['text'] = df.apply(get_text, axis=1)
df['QuestionId_Answer'] = df['QuestionId'].astype(str) + '_' + df['AnswerId'].astype(str)

In [6]:
print(df['text'][0])

Instruct: Given a math question and a misconcepte incorrect answer, please retrieve the most accurate reason for the misconception.

Query:
###Construct###:Use the order of operations to carry out calculations involving powers
###Subject###:BIDMAS
###Question###:\[
3 \times 2+4-5
\]
Where do the brackets need to go to make the answer equal \( 13 \) ?
###Correct Answer###:\( 3 \times(2+4)-5 \)
###Incorrect Answer###:\( 3 \times 2+(4-5) \)
###Misconception###:


In [7]:
len(df)

9

In [8]:
df.head(3)

,QuestionId,CorrectAnswer,ConstructName,SubjectName,QuestionText,Answer,AnswerText,AnswerId,CorrectAnswerText,text,QuestionId_Answer
0,1869,A,Use the order of operations to carry out calcu...,BIDMAS,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,AnswerBText,\( 3 \times 2+(4-5) \),B,\( 3 \times(2+4)-5 \),Instruct: Given a math question and a misconce...,1869_B
1,1869,A,Use the order of operations to carry out calcu...,BIDMAS,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,AnswerCText,\( 3 \times(2+4-5) \),C,\( 3 \times(2+4)-5 \),Instruct: Given a math question and a misconce...,1869_C
2,1869,A,Use the order of operations to carry out calcu...,BIDMAS,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,AnswerDText,Does not need brackets,D,\( 3 \times(2+4)-5 \),Instruct: Given a math question and a misconce...,1869_D


In [9]:
df.to_parquet("df_preprocessed.parquet", index=False)

# Embedder

In [10]:
%%writefile run_embedder.py

import argparse
from tqdm import tqdm, trange
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import torch
import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, Qwen2Model, BitsAndBytesConfig, set_seed
from peft import LoraConfig, get_peft_model

# パラメータ
MODEL_PATH = "/kaggle/input/qwen2.5-32b/transformers/default/1"
MAX_LENGTH = 512
BATCH_SIZE = 8
DEVICE = 'auto'

set_seed(42)

def last_token_pool(last_hidden_states: Tensor, attention_mask: Tensor) -> Tensor:
    """
    Extract the embedding of the last valid token based on attention mask.
    """
    if attention_mask[:, -1].all():  # Check if right padding is used
        return last_hidden_states[:, -1]
    sequence_lengths = attention_mask.sum(dim=1) - 1
    return last_hidden_states[torch.arange(last_hidden_states.size(0)), sequence_lengths]

@torch.no_grad()
@torch.amp.autocast('cuda')
def inference(df, model, tokenizer, query_text='query_text'):
    """
    Perform inference to extract embeddings for a given dataframe of sentences.
    """
    sentences = df[query_text].tolist()
    all_embeddings = [None] * len(sentences)

    # Sort sentences by length (descending)
    length_sorted_idx = np.argsort([-len(sen) for sen in sentences])
    sentences_sorted = [sentences[idx] for idx in length_sorted_idx]

    for start_idx in trange(0, len(sentences), BATCH_SIZE, desc="Batches"):
        sentences_batch = sentences_sorted[start_idx: start_idx + BATCH_SIZE]
        features = tokenizer(sentences_batch, max_length=MAX_LENGTH, padding=True, truncation=True, return_tensors="pt")
        features = {key: value.to(model.device) for key, value in features.items() if isinstance(value, Tensor)}

        outputs = model.model(**features)
        embeddings = last_token_pool(outputs.last_hidden_state, features['attention_mask'])
        embeddings = torch.nn.functional.normalize(embeddings, dim=-1).cpu().numpy()

        for i, emb in enumerate(embeddings):
            original_idx = length_sorted_idx[start_idx + i]
            all_embeddings[original_idx] = emb

    return np.vstack(all_embeddings)

if __name__ == "__main__":
    # Parse arguments
    parser = argparse.ArgumentParser(description="Run embedding inference.")
    parser.add_argument("--df_path", type=str, required=True)
    parser.add_argument("--lora_path", type=str, required=True)
    parser.add_argument("--output_path", type=str, required=True)
    args = parser.parse_args()

    # Load data
    df_path = args.df_path
    lora_path = args.lora_path
    output_path = args.output_path

    df = pd.read_parquet(df_path)
    misconceptions = pd.read_csv('/kaggle/input/eedi-mining-misconceptions-in-mathematics/misconception_mapping.csv')

    # Load tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
    bnb_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_use_double_quant=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_compute_dtype=torch.bfloat16
            )
    model = Qwen2Model.from_pretrained(
        MODEL_PATH,
        quantization_config=bnb_config,
        device_map=DEVICE,
    )
    config = LoraConfig(
            r=64,
            lora_alpha=128,
            target_modules=[
                "q_proj",
                "k_proj",
                "v_proj",
                "o_proj",
                "gate_proj",
                "up_proj",
                "down_proj",
            ],
            bias="none",
            lora_dropout=0.05,
            task_type="CAUSAL_LM",
        )
    model = get_peft_model(model, config)
    d = torch.load(lora_path, map_location=model.device, weights_only=True)
    model.load_state_dict(d, strict=False)

    # Perform inference
    query_embeddings = inference(df, model, tokenizer, query_text='text')
    passage_embeddings = inference(misconceptions, model, tokenizer, query_text='MisconceptionName')

    # Compute similarity and save results
    similarity = cosine_similarity(query_embeddings, passage_embeddings)
    np.save(output_path, similarity)

Writing run_embedder.py


In [11]:
!python run_embedder.py \
    --df_path df_preprocessed.parquet \
    --lora_path /kaggle/input/embedder-qwen2.5-32b-lora-v2/transformers/default/1/adapter.bin \
    --output_path similarity1.npy

Batches: 100%|████████████████████████████████| 324/324 [16:12<00:00,  3.00s/it]


In [12]:
!python run_embedder.py \
    --df_path df_preprocessed.parquet \
    --lora_path /kaggle/input/embedder-lora-v6/transformers/default/1/adapter.bin \
    --output_path similarity2.npy

Batches: 100%|████████████████████████████████| 324/324 [16:12<00:00,  3.00s/it]


In [13]:
sim1 = np.load('similarity1.npy')
sim2 = np.load('similarity2.npy')

In [14]:
for i in range(sim1.shape[0]):
    sim1[i, zeros] *= 1.5
    sim2[i, zeros] *= 1.5

In [15]:
top_n_indices1 = np.argsort(sim1, axis=1)[:, ::-1][:, :25]
top_n_indices2 = np.argsort(sim2, axis=1)[:, ::-1][:, :25]

In [16]:
df = pd.read_parquet('df_preprocessed.parquet')

df['top_indices1'] = top_n_indices1.tolist()
df['top_indices2'] = top_n_indices2.tolist()
df.to_parquet('df_embedded.parquet', index=False)

# Reranker 1

In [17]:
import re

def get_candidates(c_indices):
    candidates = []

    mis_names = misconceptions["MisconceptionName"].values
    for ix in c_indices:
        c_names = []
        for i, name in enumerate(mis_names[ix]):
            # A, B, C...形式に変更
            c_names.append(f"{chr(65 + i)}. {name}")

        candidates.append("\n".join(c_names))

    return candidates

def preprocess_text(x):
    x = re.sub(r"http\w+", '', x)   # Delete URL
    x = re.sub(r"\.+", ".", x)      # Replace consecutive periods with a single period
    x = re.sub(r"\,+", ",", x)      # Replace consecutive commas with a single comma
    x = re.sub(r"\\\\", r"\\", x)   # Normalize multiple backslashes to double backslashes
    x = re.sub(r"[ ]{2,}", " ", x)  # Replace multiple spaces with a single space
    x = x.strip()                   # Remove empty characters at the beginning and end
    return x

PROMPT = """Here is a question about {ConstructName}({SubjectName}).
Question: {Question}
Correct Answer: {CorrectAnswer}
Incorrect Answer: {IncorrectAnswer}

You are a Mathematics teacher. Your task is to reason and identify the misconception behind the Incorrect Answer with the Question.
Choose the most appropriate letter corresponding to the misconception from the options below:

{Retrival}
"""

def preprocess_row(row):
    conversations = [
        {
            "role": "user",
            "content": preprocess_text(
                PROMPT.format(
                    ConstructName=row["ConstructName"],
                    SubjectName=row["SubjectName"],
                    Question=row["QuestionText"],
                    CorrectAnswer=row["CorrectAnswerText"],
                    IncorrectAnswer=row["AnswerText"],
                    Retrival=row["retrieval"]
                )
            )
        }
    ]
    return conversations

In [18]:
def process_dataframe(top_indices_column, output_file, top_k):
    df = pd.read_parquet('df_embedded.parquet')
    top_indices = np.array(df[top_indices_column].tolist())[:, :top_k]
    df['retrieval'] = get_candidates(top_indices)
    df['conversations'] = df.apply(preprocess_row, axis=1)
    df['TOP_K'] = top_k
    df['id'] = range(len(df))
    df.to_parquet(output_file, index=False)
    print(df['conversations'][0][0]['content'])
    print('-'*50)

# 定数の設定
TOP_K = 25

# ファイル処理の呼び出し
process_dataframe('top_indices1', 'df_test1.parquet', TOP_K)
process_dataframe('top_indices2', 'df_test2.parquet', TOP_K)

Here is a question about Use the order of operations to carry out calculations involving powers(BIDMAS).
Question: \[
3 \times 2+4-5
\]
Where do the brackets need to go to make the answer equal \( 13 \) ?
Correct Answer: \( 3 \times(2+4)-5 \)
Incorrect Answer: \( 3 \times 2+(4-5) \)

You are a Mathematics teacher. Your task is to reason and identify the misconception behind the Incorrect Answer with the Question.
Choose the most appropriate letter corresponding to the misconception from the options below:

A. Inserts brackets but not changed order of operation
B. May have made a calculation error using the order of operations
C. Carries out operations from left to right regardless of priority order, unless brackets are used
D. Confuses the order of operations, believes subtraction comes before multiplication 
E. Has removed brackets but not performed the operation
F. Has not realised that the answer may be changed by the insertion of brackets
G. Thinks the subtraction sign means multip

In [19]:
%%writefile run_reranker1.py

import argparse
import time
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

import random
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, set_seed
from transformers.data.data_collator import pad_without_fast_tokenizer_warning
from peft import PeftModel

# 固定パラメータ
MODEL_DIR = '/kaggle/input/qwen2-5-32b-instruct-bnb-4bit'
MAX_LENGTH = 1536
BATCH_SIZE = 1

set_seed(42)

def tokenize(tokenizer, conversations, max_length=MAX_LENGTH):
    texts = []
    for messages in conversations:
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        texts.append(text)

    tokenized = tokenizer(texts, add_special_tokens=False, max_length=max_length, truncation=True)
    input_ids = tokenized.input_ids
    attention_mask = tokenized.attention_mask
    return input_ids, attention_mask

@torch.no_grad()
@torch.amp.autocast('cuda')
def inference(df, model, label_idx, batch_size=BATCH_SIZE, max_length=MAX_LENGTH):
    scores = []

    # バッチごとに処理を行うループ
    for start_idx in tqdm(range(0, len(df), batch_size)):
        end_idx = min(start_idx + batch_size, len(df))
        tmp = df.iloc[start_idx:end_idx]
        input_ids = tmp["input_ids"].to_list()
        attention_mask = tmp["attention_mask"].to_list()
        
        # 入力のパディング処理
        inputs = pad_without_fast_tokenizer_warning(
            tokenizer,
            {"input_ids": input_ids, "attention_mask": attention_mask},
            padding="longest",
            pad_to_multiple_of=None,
            return_tensors="pt",
        )
        
        # モデル推論
        outputs = model(**inputs.to(model.device))
        proba = outputs.logits[:, -1, label_idx].softmax(-1).cpu()
        scores.extend(proba.tolist())
        
    df['score'] = scores
    return df

if __name__ == "__main__":
    # コマンドライン引数を定義
    parser = argparse.ArgumentParser(description="Run reranker inference.")
    parser.add_argument("--lora_dir1", type=str, required=True)
    parser.add_argument("--input_path", type=str, required=True)
    parser.add_argument("--output_path", type=str, required=True)
    args = parser.parse_args()

    # 設定
    lora_dir1 = args.lora_dir1
    input_path = args.input_path
    output_path = args.output_path

    # トークナイザーとモデルの読み込み
    tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_DIR,
        device_map='auto',
        use_cache=True,
    )

    # LoRAアダプターのロード
    model = PeftModel.from_pretrained(model, lora_dir1, adapter_name="lora1")

    # データ読み込み
    df_test = pd.read_parquet(input_path)
    TOP_K = df_test['TOP_K'][0]
    label_idx = [tokenizer(f'{chr(65 + i)}', add_special_tokens=False)['input_ids'][-1] for i in range(TOP_K)]
    print(tokenizer.decode(label_idx))

    # データのトークン化
    data = pd.DataFrame()
    data["id"] = df_test["id"]
    data["input_ids"], data["attention_mask"] = tokenize(tokenizer, df_test["conversations"])
    data["length"] = data["input_ids"].apply(len)
    data = data.sort_values("length", ascending=False).reset_index(drop=True)

    # 推論と保存
    score_df = inference(data, model, label_idx).sort_values('id').reset_index(drop=True)
    score_df.to_parquet(output_path, index=False)

Writing run_reranker1.py


In [20]:
!python run_reranker1.py \
    --lora_dir1 /kaggle/input/reranker1-lora-v33/transformers/default/1 \
    --input_path df_test1.parquet \
    --output_path score_df1.parquet

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
Loading checkpoint shards: 100%|██████████████████| 4/4 [02:29<00:00, 37.45s/it]
ABCDEFGHIJKLMNOPQRSTUVWXY
100%|█████████████████████████████████████████████| 9/9 [00:36<00:00,  4.02s/it]


In [21]:
!python run_reranker1.py \
    --lora_dir1 /kaggle/input/reranker1-lora-v65/transformers/default/1 \
    --input_path df_test2.parquet \
    --output_path score_df2.parquet

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
Loading checkpoint shards: 100%|██████████████████| 4/4 [00:11<00:00,  2.95s/it]
ABCDEFGHIJKLMNOPQRSTUVWXY
100%|█████████████████████████████████████████████| 9/9 [00:36<00:00,  4.01s/it]


# Submit

In [22]:
df_embedded = pd.read_parquet('df_embedded.parquet')
score_df1 = pd.read_parquet("score_df1.parquet")
score_df2 = pd.read_parquet("score_df2.parquet")

In [23]:
# top_indices1 = np.array(df_embedded['top_indices1'].tolist())
# top_indices2 = np.array(df_embedded['top_indices2'].tolist())

# scores1 = np.array(score_df1['score'].tolist())
# scores2 = np.array(score_df2['score'].tolist())

# # 配列を結合
# indices_combined = np.concatenate([top_indices1, top_indices2], axis=1)  # 2次元で列方向に結合
# scores_combined = np.concatenate([scores1, scores2], axis=1)  # 同じく列方向に結合

# # 各行でスコア降順にソート
# sorted_indices = np.argsort(-scores_combined, axis=1)  # スコアの降順ソートインデックス
# sorted_top_indices = np.take_along_axis(indices_combined, sorted_indices, axis=1)

# # 重複を削除して最終結果を取得
# final_indices = []
# for row in sorted_top_indices:
#     unique_row = np.unique(row, return_index=True)[1]  # 重複を除外
#     final_indices.append(row[np.sort(unique_row)][:25])  # 元の順序を維持

# final_indices = np.array(final_indices).tolist()

In [24]:
top_indices1 = np.array(df_embedded['top_indices1'].tolist())
top_indices2 = np.array(df_embedded['top_indices2'].tolist())

scores1 = np.array(score_df1['score'].tolist())
scores2 = np.array(score_df2['score'].tolist())

# それぞれの行ごとに処理
final_indices = []
for i in range(len(top_indices1)):
    # 現行の行のindexとスコア
    row_indices1 = top_indices1[i]
    row_scores1 = scores1[i]
    row_indices2 = top_indices2[i]
    row_scores2 = scores2[i]

    # インデックスとスコアをまとめるための辞書 (index -> score)
    combined_dict = {}

    # indices1, scores1を追加
    for idx, s in zip(row_indices1, row_scores1):
        combined_dict[idx] = s

    # indices2, scores2を追加。既に存在するindexなら平均化
    for idx, s in zip(row_indices2, row_scores2):
        if idx in combined_dict:
            combined_dict[idx] = (combined_dict[idx] + s) / 2.0
        else:
            combined_dict[idx] = s

    # `zeros` に含まれるインデックスのスコアを 1.3 倍
    for idx in zeros:
        if idx in combined_dict:
            combined_dict[idx] *= 1.3

    # 辞書から(インデックス, スコア)タプルのリストを作成し、スコア降順にソート
    combined_list = sorted(combined_dict.items(), key=lambda x: x[1], reverse=True)

    # 上位25件を取得
    top_25_indices = [x[0] for x in combined_list[:25]]
    final_indices.append(top_25_indices)

In [25]:
submission = df_embedded.copy()
submission['MisconceptionId'] = final_indices

In [26]:
submission["MisconceptionId"] = submission["MisconceptionId"].apply(lambda x: ' '.join(map(str, x)))
submission[['QuestionId_Answer', 'MisconceptionId']].to_csv('submission.csv', index=False)

In [27]:
submission[['QuestionId_Answer', 'MisconceptionId']]

,QuestionId_Answer,MisconceptionId
0,1869_B,1005 1345 2518 1507 315 1941 2306 1672 2532 70...
1,1869_C,1507 1345 1005 315 2518 2306 2488 1941 2449 24...
2,1869_D,1507 1005 2532 2518 1672 328 1941 315 2306 139...
3,1870_A,1755 891 1593 167 1535 1991 1191 59 885 683 22...
4,1870_B,2398 363 1755 143 891 1991 1904 1593 59 885 24...
5,1870_C,1755 891 167 2277 1191 1593 59 1991 683 320 88...
6,1871_A,1287 1059 1073 632 2064 1797 2439 1200 2471 21...
7,1871_C,1287 1073 1059 2439 2471 632 1200 2151 1797 36...
8,1871_D,1073 1059 1287 2151 2471 1200 632 1094 1797 21...
